In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from PIL import Image
from keras.models import Sequential
from keras.layers import MaxPooling2D,Flatten,Dense,Conv2D,Activation,Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from sklearn.metrics import classification_report,confusion_matrix
import tensorflow as tf
import cv2
import os
import numpy as np
from keras.layers.normalization import BatchNormalization
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import pandas as pd


Using TensorFlow backend.


In [2]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(

    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.3,
    zoom_range=0.3,
    fill_mode="nearest",
    cval=0.0,
    horizontal_flip=True,
    vertical_flip=True,
    rescale = 1./255,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0,
    dtype=None,
)

In [5]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('C:/Users/ADMIN PC/Downloads/medicina Leaf project/medicine/train', class_mode='categorical', target_size=(150, 150))
# load and iterate test dataset
test_it = datagen.flow_from_directory('C:/Users/ADMIN PC/Downloads/medicina Leaf project/medicine/test', class_mode='categorical', target_size=(150, 150))

Found 418 images belonging to 12 classes.
Found 322 images belonging to 13 classes.


In [6]:
## model architecture
model = tf.keras.models.Sequential([
    # first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation="relu", input_shape=(150,150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    # second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    # third convolution layer
    tf.keras.layers.Conv2D(128, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    # fourth convolution layer
    tf.keras.layers.Conv2D(128, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    # flatten before feeding into Dense neural network. 
    tf.keras.layers.Flatten(),
    # 512 neurons in the hidden layer
    tf.keras.layers.Dense(512, activation="relu"),
    # 15 = 15 different categories
    # softmas takes a set of values and effectively picks the biggest one. for example if the output layer has
    # [0.1,0.1,0.5,0.2,0.1], it will take it and turn it into [0,0,1,0,0]
    tf.keras.layers.Dense(12, activation="softmax")
]);

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

In [13]:
# implementing a callback function to terminate training once training reaches 98% accuracy for validation data

validation_accuracy = 0.98

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get("val_accuracy") >= validation_accuracy):
            print("\nReached desired validation accuracy, so cancelling training")
            self.model.stop_training=True
            
callbacks = myCallback()

In [14]:
model.compile(loss = "categorical_crossentropy", optimizer='rmsprop', metrics=['accuracy'])

In [11]:
med_model = model.fit_generator(train_it,epochs=50, validation_data=test_it, verbose=1)

filepath = r"/content/drive/MyDrive/savedmodeleaves"
tf.keras.models.save_model(
    model,
    filepath,
    overwrite=True,
    include_optimizer=True,
    save_format="tf",
    signatures=None
)

model.save("medgen.h5")

Instructions for updating:
Please use Model.fit, which supports generators.


Instructions for updating:
Please use Model.fit, which supports generators.


  ...
    to  
  ['...']


  ...
    to  
  ['...']


  ...
    to  
  ['...']


  ...
    to  
  ['...']


Train for 14 steps, validate for 11 steps
Epoch 1/50
14/14 [==============================] - 53s 4s/step - loss: 2.4837 - accuracy: 0.1244


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\ADMIN PC\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-8f0684a538bb>", line 1, in <module>
    med_model = model.fit_generator(train_it,epochs=50, validation_data=test_it, verbose=1)
  File "C:\Users\ADMIN PC\Anaconda3\lib\site-packages\tensorflow_core\python\util\deprecation.py", line 324, in new_func
    return func(*args, **kwargs)
  File "C:\Users\ADMIN PC\Anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\training.py", line 1306, in fit_generator
    initial_epoch=initial_epoch)
  File "C:\Users\ADMIN PC\Anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\training.py", line 819, in fit
    use_multiprocessing=use_multiprocessing)
  File "C:\Users\ADMIN PC\Anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\training_v2.py", line 395, in fit
    total_epochs=1)
  File "C:\Users

InvalidArgumentError:  logits and labels must be broadcastable: logits_size=[32,12] labels_size=[32,13]
	 [[node loss/dense_1_loss/softmax_cross_entropy_with_logits (defined at <ipython-input-11-8f0684a538bb>:1) ]] [Op:__inference_distributed_function_1401]

Function call stack:
distributed_function


In [12]:
# loading saved model for verification. 
# loading using two methods. One is using .h5 file and loading using file path. 

# loading using .h5 file
new_med_model = tf.keras.models.load_model(
    "medgen.h5",
    custom_objects=None,
    compile=True
)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\ADMIN PC\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-b891bccb801d>", line 8, in <module>
    compile=True
  File "C:\Users\ADMIN PC\Anaconda3\lib\site-packages\tensorflow_core\python\keras\saving\save.py", line 149, in load_model
    loader_impl.parse_saved_model(filepath)
  File "C:\Users\ADMIN PC\Anaconda3\lib\site-packages\tensorflow_core\python\saved_model\loader_impl.py", line 83, in parse_saved_model
    constants.SAVED_MODEL_FILENAME_PB))
OSError: SavedModel file does not exist at: medgen.h5/{saved_model.pbtxt|saved_model.pb}

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\ADMIN PC\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' ob

OSError: SavedModel file does not exist at: medgen.h5/{saved_model.pbtxt|saved_model.pb}

In [ ]:
new_med_model.summary()

In [ ]:
model_dir = r"/content/drive/MyDrive/savedmodeleaves"

# loading using .h5 file
new_med_model_v2 = tf.keras.models.load_model(
    model_dir,
    custom_objects=None,
    compile=True
)

In [ ]:
new_med_model_v2.summary()

In [ ]:

from keras_preprocessing import image
labels = ['Alpinia Galanga (Rasna)', 'Citrus Limon (Lemon)','Ficus Religiosa (Peepal Tree)','Hibiscus Rosa-sinensis','Mentha (Mint)','Moringa Oleifera (Drumstick)','Murraya Koenigii (Curry)','Ocimum Tenuiflorum (Tulsi)','Psidium Guajava (Guava)','Punica Granatum (Pomegranate)','Santalum Album (Sandalwood)','Syzygium Cumini (Jamun)']
image_dir = r"/content/drive/MyDrive/pre"
img_list = os.listdir(image_dir)

for leaves in img_list:
    path = os.path.join(image_dir, leaves)
    img = image.load_img(path, target_size = (150, 150))
    array = image.img_to_array(img)
    x = np.expand_dims(array, axis=0)

    vimage = np.vstack([x])
    img_classification = model.predict(vimage)
    y_classes = np.argmax(img_classification)
    print(labels[y_classes])


In [ ]:
  img = []
  images = image.load_img("/content/drive/MyDrive/pre/OT-S-047.jpg", target_size=(150,150))    
  x = image.img_to_array(images)
  x = tf.image.rgb_to_grayscale(x)
  x = np.expand_dims(x, axis=0)
  x = x/255.0
  predy = (model.predict(x) > 0.5).astype("int32")
  y_classes = np.argmax(to_categorical(predy))
  print(categories[y_classes])

In [ ]:
loss,acc = new_med_model.evaluate(test_it, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

In [ ]:
pd.DataFrame(med_model.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) 
plt.show()